## エンベディング
1. 数値への変換の仕方
2. 数値の表現

In [2]:
import os
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
)

from dotenv import load_dotenv
load_dotenv('../.env')

True

### 1.数値への変換の仕方
 - <a href="https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.azure.AzureOpenAIEmbeddings.html" target=_blank>ドキュメント1</a>
 - <a href="https://python.langchain.com/v0.2/docs/integrations/text_embedding/azureopenai/" target=_blank>ドキュメント2</a>

In [3]:
## モデルの取得
# 3. OpenAIのembeddingモデル取得
embeddings = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-3-small", # Azureのデプロイメント名
        openai_api_version="2023-05-15", # OpenAIのAPIバージョン
 #       azure_endpoint='https://your-azure-openai-endpoint.openai.azure.com/',
    )
elif os.getenv('OPENAI_API_KEY') != "":
    # OpenAIの場合
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
else:
    print("APIKeyの設定を確認してください")

#### 1-2.数値取得

In [4]:
text = "これはテスト用のドキュメントです。"

In [5]:
# ひとつベクトル化し、ベクトルの次元を確認する
query_result = embeddings.embed_query(text)
print(f"サイズ:{len(query_result)}")
print(f"5のみ確認")
print(query_result[:5])

サイズ:1536
5のみ確認
[0.006708289030939341, 0.06086710840463638, -0.008633030578494072, -0.04146118834614754, 0.026652004569768906]


In [6]:
# 配列で処理
doc_result = embeddings.embed_documents([text])
print(f"サイズ:{len(doc_result[0])}")
print(f"5のみ確認")
print(doc_result[0][:5])

サイズ:1536
5のみ確認
[0.006708289030939341, 0.06086710840463638, -0.008633030578494072, -0.04146118834614754, 0.026652004569768906]


### 2. 数値の表現

In [25]:
perplexity = 2
tSNE_text_font_size = 10
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objs as go
import plotly.express as px
import os
import pandas as pd

In [26]:
texts = [
    "カレーたべたい",
    "カツカレー食べたい",
    "うどん食べたい",
    "カレーを作る",
    "パエリアーを作る",
    "明日の天気は",
    "雨だから傘を持っていく",
    "晴れたらプールいく",
    "雨なので家で遊ぶ",
    "プログラムを勉強する",
    "Pythonを勉強する",
    "AIを使ったのアプリ開発",
    "PythonでAIを実装",
]
doc_result = embeddings.embed_documents(texts)
doc_result = np.array(doc_result)
doc_result.shape


(13, 1536)

In [27]:
# 実行
tsne = TSNE(n_components=2, perplexity=perplexity, random_state=0)
np.set_printoptions(suppress=True)
value2 = tsne.fit_transform(doc_result)

In [28]:
# 可視化
fig2 = go.Figure()
for value, word in zip(value2, texts):
    fig2.add_trace(
        go.Scatter(
            x = pd.Series(value[0]),
            y = pd.Series(value[1]),
            mode = 'markers+text',
            text= word,
            textposition= 'top center',
        )
    )

In [29]:
fig2.update_layout(title=dict(text='embeddingsの可視化',
                              font=dict(size=18,
                                        color='grey'),
                              xref='paper', # container or paper
                              x=0.5,
                              y=0.9,
                              xanchor='center'
                              ),
                    showlegend=False,
                    font = dict(size=10),
                    width=700,
                    height=650)
fig2.show()
